In [ ]:
#Are there patterns over time in non-member and member attendance based on zip code and demographics? (Month to month)
    #Exploratory

#What are membership renewal rates? How many times are guests returning before becoming members?
    #Probably can't answer this question with this data. Data shows membership re-entry, but does not capture return General Admission.

#Model predicted non-member/member attendance considering population increase. What might attendance be like in 10 years?
    #Would need to tie in projected population by county/zip code from another data source. Maybe just focus on this data set for now, tackle population projection if there's time
    ##For modeL:
    ##target variable would be 'member'
    ##assign 1 as member (accesscodegroup '3 Members') and 0 as non-member (accesscodegroup '1 GA')

#What cities should marketing efforts focus on (proportionately to population) to improve guest attendance?

In [ ]:
# Utah population projections by county: https://tableau.dashboard.utah.edu/t/Business/views/20220111_Detailed_Proj_Vis/ProjectionsWorkbook?%3Adisplay_count=n&%3Aembed=y&%3AisGuestRedirectFromVizportal=y&%3Aorigin=viz_share_link&%3AshowAppBanner=false&%3AshowVizHome=n

In [56]:
# Import Dependencies
import pandas as pd
import numpy as np

In [57]:
#Load and read original csv data file
attendance_data_csv = "./Resources/AttendanceATSani.csv"

attendance_df = pd.read_csv(attendance_data_csv)
attendance_df.head()

C:\Users\camih\AppData\Local\Temp\ipykernel_7592\1484746679.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  attendance_df = pd.read_csv(attendance_data_csv)


,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup
0,1,2/22/22 0:00,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
1,1,2/22/22 0:00,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
2,1,2/22/22 0:00,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
3,0,2/22/22 0:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000004,2/22/22 14:04,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis
4,0,2/22/22 0:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000005,2/22/22 14:04,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis


In [58]:
#Count rows of original data
row_count = len(attendance_df)
row_count

1048575

In [59]:
#Check data types of original data
attendance_df.dtypes

admits              int64
usedate            object
plu                object
pluname            object
accesscode          int64
accesscodegroup    object
accesscodename     object
usageid             int64
saledatetime       object
salenode            int64
zipcode            object
city               object
county             object
stateloc           object
subgroup           object
maingroup          object
dtype: object

In [60]:
#Drop all rows containing missing or NULL zipcode values
attendance_df.dropna(subset=['zipcode'], inplace=True)

attendance_df = attendance_df[attendance_df['zipcode'] != '     ']

#clean_attendance_df = attendance_df.dropna(inplace = True)

In [61]:
#Count rows of data after dropping missing and NULL zipcodes. Note data loss in printed message
dropna_row_count = len(attendance_df)
dropna_row_count
print(f"Data loss from dropped rows containing missing/NULL zipcodes: {row_count - dropna_row_count}")

Data loss from dropped rows containing missing/NULL zipcodes: 103787


In [62]:
clean_attendance_df = attendance_df[~attendance_df['zipcode'].isnull()]
clean_attendance_df.head()

,admits,usedate,plu,pluname,accesscode,accesscodegroup,accesscodename,usageid,saledatetime,salenode,zipcode,city,county,stateloc,subgroup,maingroup
0,1,2/22/22 0:00,GA01010TDP,Adult Admission,108,1 GA,Timed Ticketing GA,10000000,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
1,1,2/22/22 0:00,GA01055TDP,Teen Admission,108,1 GA,Timed Ticketing GA,10000002,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
2,1,2/22/22 0:00,GA01060T,2 & Under Admission,108,1 GA,Timed Ticketing GA,10000003,2/22/22 13:46,406,84129,SALT LAKE CITY,Salt Lake,UT,"Salt Lake City, Taylorsville",Salt Lake County
3,0,2/22/22 0:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000004,2/22/22 14:04,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis
4,0,2/22/22 0:00,TI0003,Tuki's Island Member,450,9 Fun Center,Tuki's Island,10000005,2/22/22 14:04,32,84010,BOUNTIFUL,Davis,UT,Bountiful,Davis


In [63]:
#Method that drops rows in the DataFrame where the specified column contains special characters
def drop_rows_with_special_chars(clean_attendance_df, zipcode):
    
    # Define a pattern for special characters
    pattern = r'[^\w\s]' 

    # Filter the DataFrame to keep only rows without special characters
    return clean_attendance_df[~clean_attendance_df[zipcode].str.contains(pattern, regex=True, na=False)]

clean_attendance_df = drop_rows_with_special_chars(clean_attendance_df, 'zipcode')

In [64]:
#Convert zipcode object datatype to integer
clean_attendance_df['zipcode'] = clean_attendance_df['zipcode'].astype(float)

clean_attendance_df['zipcode'] = np.ceil(clean_attendance_df['zipcode']).astype(int)

In [68]:
#Count rows of data after dropping zipcodes containing special characters. Note further data loss in printed message
clean_row_count = len(clean_attendance_df)
clean_row_count

print(f"Further data loss from dropped rows with zipcodes containing special characters: {dropna_row_count - clean_row_count}")

Further data loss from dropped rows with zipcodes containing special characters: 9


In [69]:
#Replace all NULL values in the dataframe with 'unknown'
clean_attendance_df.fillna('unknown', inplace=True)

In [70]:
#Replace 'Blank ZIP Field', 'Uncaptured Zip Codes' with 'unknown' and blank values with 0
clean_attendance_df = clean_attendance_df.replace({'Blank ZIP Field':'unknown', 'Uncaptured Zip Codes': 'unknown', '':'0'})

In [71]:
#Filter out 'unknown' saledatetimes out of the dataframe
clean_attendance_df = clean_attendance_df[~clean_attendance_df['saledatetime'].str.contains('unknown', na=False)]

In [72]:
#Count rows of data after filtering out rows with 'unknown' saledatetimes. Note further data loss in printed message
cleanest_row_count = len(clean_attendance_df)
cleanest_row_count

print(f"Data loss from dropped rows containing unknown saledatetimes: {clean_row_count - cleanest_row_count}")

Data loss from dropped rows containing missing/NULL zipcodes: 877


In [76]:
#Print total data loss from cleaning
print(f"Total data loss from cleaning: {row_count - cleanest_row_count}. Total remaining data row count: {cleanest_row_count}")

Total data loss from cleaning: 104673. Total remaining data row count: 943902


In [75]:
clean_attendance_df.plu.unique()

array(['GA01010TDP          ', 'GA01055TDP          ',
       'GA01060T            ', 'TI0003              ',
       'GA01050TDP          ', 'MB00001             ',
       'MB00011             ', 'TI0002              ',
       'MB00057             ', 'MB00013             ',
       'MB00057R            ', 'MB00015             ',
       'MPR01               ', 'GA01030TDP          ',
       'MPR21               ', 'MB00014-15          ',
       'GA01020TDP          ', 'MB00009             ',
       'GA01040TDP          ', 'MB00012-15          ',
       'TI0001              ', 'MB00057BF           ',
       'COMP04001           ', 'MB00002-15          ',
       'MPR05               ', 'GA01110             ',
       'MPR22               ', 'MPR10               ',
       'MB00055R            ', 'SCTF00001           ',
       'SCTF00001M          ', 'MB00005             ',
       'MB00016-15          ', 'MPR04               ',
       'MB00013BF           ', 'MB00003BF           ',
       'MB

In [77]:
clean_attendance_df.pluname.unique()

array(['Adult Admission', 'Teen Admission', '2 & Under Admission',
       "Tuki's Island Member", 'Child Admission', 'Child Membership',
       'Adult Membership', "Tuki's Island 2 & Under",
       'Quad Family Membership', 'Family Membership',
       'Quad Family Membership Renewal', 'Family Plus Membership',
       'Family Membership Promo', 'Military Admission',
       'Quad Membership Promo', 'Family Membership 4 Child Renewal',
       'Senior Admission', 'Senior Membership', 'Student Admission',
       'Adult Membership Renewal', "Tuki's Island 1 Hr",
       'Quad Family Membership Black Friday', 'All Ages VIP',
       'Child Membership Renewal', 'Adult Membership Promo',
       'Member Promo GA Admission', 'Child Membership Promo',
       'Grandparent Membership Renewal', 'Teen STEM Futures',
       'Student Membership', 'Family Plus Membership Renewal',
       'Grandparent Plus Promo', 'Family Membership Black Friday',
       'Teen Membership Black Friday', 'Adult Membership Bla

In [78]:
clean_attendance_df.accesscode.unique()

array([  108,   450,    11,    21,    20,    10,   150,   100,    24,
         415,   310,   311,    23,   210,   200,   502,   501,    31,
         106,    30, 91000,   411,   420,   107,   412,   300, 12550,
         410,   111,    26,   406,   401,  3000,   112,   414, 95110,
       20200,   416,   220, 91001,   413,   407], dtype=int64)

In [79]:
clean_attendance_df.accesscodegroup.unique()

array(['1 GA', '9 Fun Center', '3 Members', '4 Comp', '9 Camp', '2 Group',
       '5 School', '8 EN', 'CHARITABLE', '6 Event', '7 Fac Rental',
       'FAM MEM', 'PARTY', '10 Camp', 'MARKETING', 'VR', 'PROMOTION'],
      dtype=object)

In [80]:
clean_attendance_df.accesscodename.unique()

array(['Timed Ticketing GA            ', "Tuki's Island                 ",
       'Membership Readmit            ', 'Family Membership Readmit     ',
       'Family Membership             ', 'Membership                    ',
       'Comp                          ', 'Aquarium Admission            ',
       'Grandparent Membership Readmit', 'Youth/Family Program          ',
       'Adult                         ', 'Student                       ',
       'Grandparent Membership        ', 'Stingray Feed                 ',
       'Penguin Encounter             ', 'Volunteer Membership          ',
       'Party Admission               ', 'Employee Membership           ',
       'CHARITABLE VOUCHER            ', 'Printed Hosted Event          ',
       'Facility Rental               ', 'Member Ticketed Admission     ',
       'Aquarium Hosted Event         ', 'Group                         ',
       'MEMBERSHIP - FAMILY WEB       ', 'Hosted Event                  ',
       'Party Room       

In [81]:
clean_attendance_df.zipcode.unique()

array([84129, 84010, 84125, ..., 60655, 61284, 59904])

In [82]:
clean_attendance_df.city.unique()

array(['SALT LAKE CITY', 'BOUNTIFUL', 'PROVO', ..., 'DUSON', 'HENLAWSON',
       'TAYLOR RIDGE'], dtype=object)

In [83]:
clean_attendance_df.county.unique()

array(['Salt Lake', 'Davis', 'Utah', 'Uintah', 'Weber', 'unknown',
       'Cache', 'Summit', 'Juab', 'Morgan', 'Tooele', 'Beaver', 'Grand',
       'Wasatch', 'Box Elder', 'Iron', 'Washington', 'Daggett',
       'Duchesne', 'Sanpete', 'Sevier', 'Kane', 'Millard', 'Garfield',
       'Rich', 'Emery', 'Carbon', 'Wayne', 'San Juan', 'Piute'],
      dtype=object)

In [84]:

clean_attendance_df.stateloc.unique()

array(['UT', 'ID', 'unknown', 'CA', 'WY', 'AZ', 'MO', 'IN', 'WA', 'TX',
       'CO', 'NY', 'NM', 'FL', 'AR', 'SC', 'ME', 'RI', 'PA', 'MI', 'MT',
       'VA', 'MA', 'IL', 'MD', 'IA', 'NV', 'VT', 'NH', 'MN', 'OK', 'HI',
       'NJ', 'OH', 'WI', 'KY', 'AK', 'CT', 'GA', 'ND', 'OR', 'NE', 'TN',
       'NC', 'LA', 'DE', 'AE', 'PR', 'SD', 'KS', 'MS', 'AL', 'AP', 'DC',
       'WV', 'AA', 'VI', 'MP', 'GU'], dtype=object)

In [85]:
clean_attendance_df.subgroup.unique()

array(['Salt Lake City, Taylorsville', 'Bountiful', 'Salt Lake City',
       'Provo', 'West Jordan', 'Vernal', 'Riverton', 'Kaysville',
       'American Fork', 'Sandy', 'Midvale', 'Lehi', 'Ogden', 'Clearfield',
       'Herriman', 'Saratoga Springs', 'Outside Utah', 'Wellsville',
       'South Jordan', 'unknown', 'Draper', 'Orem', 'Park City', 'Nephi',
       'Morgan', 'Pleasant Grove', 'West Jordan, West Valley City',
       'Tooele', 'Eagle Mountain', 'Milford', 'Logan', 'Moab', 'Alpine',
       'Heber City', 'Mantua', 'Syracuse', 'Magna', 'Layton',
       'Cedar City', 'Roy', 'Springville', 'Hurricane', 'Santaquin',
       'Manila', 'Payson', 'Lindon', 'North Salt Lake', 'Hill Afb',
       'Duchesne', 'Altonah', 'Spanish Fork', 'Event, School, etc.',
       'Stockton', 'Salem', 'Altamont', 'Woods Cross', 'Tremonton',
       'Saint George', 'Talmage', 'Ephraim', 'Midway', 'Centerville',
       'Richfield', 'Rush Valley', 'Glendale', 'Delta', 'Santa Clara',
       'Ivins', 'Bryce', 'Ro

In [86]:
clean_attendance_df.maingroup.unique()

array(['Salt Lake County', 'Davis', 'Utah County', 'Other Utah Counties',
       'Out of State', 'unknown'], dtype=object)

In [87]:
clean_attendance_df.to_csv("./Resources/clean_attendance.csv", index=False)